## Extract Certain Pages from pdf

In [3]:
import PyPDF2
import ipywidgets as widgets
from IPython.display import display, FileLink
from datetime import datetime

# Global variable for output file name
DEFAULT_OUTPUT_NAME = "Output_Report"

# Function to get the current year and week
def get_year_and_week():
    """Return the current year and week of the year as strings."""
    now = datetime.now()
    year = now.strftime("%Y")         # Current year
    week_of_year = now.strftime("%U") # Week number of the year
    return year, week_of_year

# File upload widget
file_upload = widgets.FileUpload(accept='.pdf', multiple=False)
output_label = widgets.Label()
download_button = widgets.Button(description="Download PDF", disabled=True)
generate_button = widgets.Button(description="Generate PDF", disabled=True)

def output_certain_pages(reader, writer, pagelist):
    for page in pagelist:
        if len(reader.pages) >= page:
            writer.add_page(reader.pages[page-1])

# Function to extract pages
def extract_pages(uploaded_file):
    try:
        # Save uploaded file to disk
        file_name = uploaded_file.name  # Get uploaded file name
        with open(file_name, 'wb') as f:
            f.write(uploaded_file.content)
        
        # Open the PDF and extract pages
        with open(file_name, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            writer = PyPDF2.PdfWriter()
            
            # Add the 1st and 3rd pages if they exist
            output_certain_pages(reader, writer, [1, 3])

            # Ensure the output file name ends with .pdf
            year, week = get_year_and_week()
            output_file_name = f"{year}-{week}-Operation-{DEFAULT_OUTPUT_NAME}.pdf"
            
            # Save the new PDF
            with open(output_file_name, 'wb') as output_pdf:
                writer.write(output_pdf)

            output_label.value = f"Extracted pages saved to {output_file_name}."
            
            # Make download button active
            download_button.disabled = False
            download_button.output_file = output_file_name
    except Exception as e:
        output_label.value = f"An error occurred: {e}"

# Function to handle download
def download_file(button):
    try:
        file_path = button.output_file
        display(FileLink(file_path, result_html_prefix="<b>Download:</b> "))
    except Exception as e:
        output_label.value = f"An error occurred while preparing download: {e}"

# Handle file upload
def on_upload_change(change):
    if file_upload.value:
        generate_button.disabled = False

# Handle generate button click
def on_generate_button_click(button):
    for uploaded_file in file_upload.value:
        extract_pages(uploaded_file)

# Attach file upload change event
file_upload.observe(on_upload_change, names='value')

# Attach generate button event
generate_button.on_click(on_generate_button_click)

# Attach download button event
download_button.on_click(download_file)

# Layout the widgets
display(widgets.VBox([
    widgets.Label("Upload a PDF file to extract the 1st and 3rd pages:"),
    file_upload,
    generate_button,
    output_label,
    download_button
]))
